# New Features in VeRoViz Version 0.4.3

Released in December, 2020

- This notebook highlights some of the new functionality enabled in v0.4.3.

---

In [1]:
import veroviz as vrv
vrv.checkVersion()

'Your current installed version of veroviz is 0.4.3, the latest version available is 0.4.2. To update to the latest version, type `pip install --upgrade veroviz` at a command-line prompt.'

--- 

## New Utility Functions
Version 0.4.3 introduces 5 new "utility" functions:
1. `assignmentsToPaths()`:  Returns a dictionary of lists of paths.  The dictionary's keys are objectIDs found in the 'assignments' dataframe.  For each 'objectID', a list is provided.  This list will contain one or more paths associated with that objectID.  If the objectID does not have any consecutive rows in the 'assignments' dataframe where the ending location differs from the starting location in the next row, then there will be only one path.  This function was created to make it easier to use functions like `closestPointLoc2Path()` and `minDistLoc2Path()`,  which require paths as inputs.
2. `arcsToPaths()`:  Like `assignmentsToPaths()`, but using an `arcs` dataframe as an input.
3. `nodesToLocs()`:  Returns a list of lists (i.e., locations) from a nodes dataframe, of the form [[lat,lon,alt], ..., [lat,lon,alt]].  
4. `closestPointLoc2Assignments()`:  Finds the point along each path of an assignments dataframe that is closest to a given location.
5. `closestPointLoc2Arcs()`:  Like `closestPointLoc2Assignments()`, but using an `arcs` dataframe as an input.

--- 

### 1. `assignmentsToPaths()`
- Returns a dictionary of lists of paths.  The dictionary's keys are objectIDs found in the 'assignments' dataframe.  For each 'objectID', a list is provided.  This list will contain one or more paths associated with that objectID.  If the objectID does not have any consecutive rows in the 'assignments' dataframe where the ending location differs from the starting location in the next row, then there will be only one path.  This function was created to make it easier to use functions like `closestPointLoc2Path()` and `minDistLoc2Path()`,  which require paths as inputs.  

In [4]:
'''
We'll begin by generating some data, 
which will be used by the `assignmentsToPaths()` function.
'''

# Specify 4 locations that could be visited:
locs = [[42.1648, -78.4293], 
        [42.1565, -78.4234], 
        [42.1443, -78.4246],
        [42.1113, -78.4212]]
        
# Now, create an `Assignments` dataframe for two vehicles 
# (a truck and a car).  

# The truck will visit nodes 1 and 2.  
exampleAssignments = vrv.createAssignmentsFromLocSeq2D(
                        locSeq         = locs[0:2],
                        serviceTimeSec = 30.0,
                        objectID       = 'Truck',
                        routeType      = 'fastest',
                        dataProvider   = 'OSRM-online',
                        leafletColor   = 'red')

# The truck will then pause 45 seconds before taking a route from node 4 to node 3. 
# There is an obvious break in these routes.  
exampleAssignments = vrv.createAssignmentsFromLocSeq2D(
                        initAssignments = exampleAssignments,
                        locSeq          = locs[-1:1:-1],
                        serviceTimeSec  = 30.0,
                        startTimeSec    = max(exampleAssignments['endTimeSec']) + 45,
                        objectID        = 'Truck',
                        routeType       = 'fastest',
                        dataProvider    = 'OSRM-online',
                        leafletColor    = 'green')

# The car will visit nodes 1 and 4.
exampleAssignments = vrv.createAssignmentsFromLocSeq2D(
                        initAssignments = exampleAssignments,
                        locSeq          = [locs[0], locs[3]],
                        serviceTimeSec  = 30.0,
                        objectID        = 'Car',
                        routeType       = 'fastest',
                        dataProvider    = 'OSRM-online',
                        leafletColor    = 'blue') 


Message: The origin point (lat: 42.1648, lon: -78.4293) is 32.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1565, lon: -78.4234) is 161.8 meters away from the road. You might find a gap between destination point and the route.

Message: The origin point (lat: 42.1113, lon: -78.4212) is 193.8 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1443, lon: -78.4246) is 39.0 meters away from the road. You might find a gap between destination point and the route.

Message: The origin point (lat: 42.1648, lon: -78.4293) is 32.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1113, lon: -78.4212) is 193.8 meters away from the road. You might find a gap between destination point and the route.


In [5]:
# Visualize the locations and assignments. 

# First, generate a `Nodes` dataframe from a list of coordinates.  
# See https://veroviz.org/docs/veroviz.generateNodes.html for other methods to generate "nodes" dataframes. 
exampleNodes = vrv.createNodesFromLocs(locs = locs)

# Now, display the nodes and assignments:
vrv.createLeaflet(nodes = exampleNodes, arcs = exampleAssignments)

In [6]:
# Generate the paths for each vehicle:
myPaths = vrv.assignmentsToPaths(assignments        = exampleAssignments, 
                                 objectID           = None, 
                                 ignoreStaticPoints = True)

- Investigate some details of the output: 

In [7]:
len(myPaths['Truck'])

2

In [8]:
myPaths['Truck']

[[[42.16451, -78.429309], [42.164482, -78.427724], [42.157619, -78.422146]],
 [[42.11118, -78.418862],
  [42.120028, -78.41856],
  [42.119975, -78.415146],
  [42.121504, -78.414822],
  [42.122706, -78.41427],
  [42.126237, -78.412131],
  [42.129462, -78.409906],
  [42.130688, -78.413378],
  [42.130762, -78.418572],
  [42.130914, -78.420397],
  [42.130984, -78.420506],
  [42.137118, -78.428013],
  [42.139694, -78.426846],
  [42.142291, -78.424425],
  [42.14424, -78.424135]]]

--- 

### 2. `arcsToPaths()`
- Like `assignmentsToPaths()`, but using an `arcs` dataframe as an input.

In [9]:
'''
We'll begin by generating some data, 
which will be used by the `arcsToPaths()` function.
'''

# Specify 4 locations that could be visited:
locs = [[42.1648, -78.4293], 
        [42.1565, -78.4234], 
        [42.1443, -78.4246],
        [42.1113, -78.4212]]

# Now, create an `Arcs` dataframe for two vehicles 
# (a truck and a car).  

# The truck will visit nodes 1 and 2.  
exampleArcs = vrv.createArcsFromLocSeq(
                 locSeq       = locs[0:2],
                 objectID     = 'Truck',
                 leafletColor = 'red')

# The truck will then take a route from node 4 to node 3. 
# There is an obvious break in these routes.  
exampleArcs = vrv.createArcsFromLocSeq(
                initArcs     = exampleArcs,
                locSeq       = locs[-1:1:-1],
                objectID     = 'Truck',
                leafletColor = 'green')

# The car will visit nodes 1 and 4.
exampleArcs = vrv.createArcsFromLocSeq(
                initArcs     = exampleArcs,
                locSeq       = [locs[0], locs[3]],
                objectID     = 'Car',
                leafletColor = 'blue') 

In [10]:
# Visualize the locations and arcs. 

# First, generate a `Nodes` dataframe from a list of coordinates.  
# See https://veroviz.org/docs/veroviz.generateNodes.html for other methods to generate "nodes" dataframes. 
exampleNodes = vrv.createNodesFromLocs(locs = locs)

# Now, display the nodes and arcs:
vrv.createLeaflet(nodes = exampleNodes, arcs = exampleArcs)

In [11]:
# Generate the paths for each vehicle:
myPaths = vrv.arcsToPaths(arcs               = exampleArcs, 
                          objectID           = None, 
                          ignoreStaticPoints = True)
myPaths

{'Truck': [[[42.1648, -78.4293], [42.1565, -78.4234]],
  [[42.1113, -78.4212], [42.1443, -78.4246]]],
 'Car': [[[42.1648, -78.4293], [42.1113, -78.4212]]]}

- Investigate some details of the output: 

In [12]:
len(myPaths['Truck'])

2

In [13]:
myPaths['Truck']

[[[42.1648, -78.4293], [42.1565, -78.4234]],
 [[42.1113, -78.4212], [42.1443, -78.4246]]]

--- 

### 3. `nodesToLocs()`
- Returns a list of lists (i.e., locations) from a nodes dataframe, of the form `[[lat,lon,alt], ..., [lat,lon,alt]]`.  

In [14]:
# Generate 4 nodes:
exampleNodes = vrv.generateNodes(
    nodeDistrib      = 'normal', 
    nodeDistribArgs  = {
        'center'         : [42.90, -78.80], 
        'stdDev'         : 10000
    },
    numNodes         = 4)

In [15]:
# Convert to locations (including altitudes in meters):
vrv.nodesToLocs(exampleNodes, includeAlt = True)

[[42.72417910395068, -78.88324010304866, 0.0],
 [42.903849633410374, -78.80150031089073, 0.0],
 [42.89830176047761, -78.8975317456136, 0.0],
 [42.91174459964476, -78.78459365957832, 0.0]]

--- 

### 4. `closestPointLoc2Assignments()`
- Finds the point along each path of an assignments dataframe that is closest to a given location.

In [16]:
'''
We'll begin by generating some data, 
which will be used by the `closestPointLoc2Assignments()` function.
'''

# Specify 4 locations that could be visited:
locs = [[42.1648, -78.4293], 
        [42.1565, -78.4234], 
        [42.1443, -78.4246],
        [42.1113, -78.4212]]
        
# Now, create an `Assignments` dataframe for two vehicles 
# (a truck and a car).  

# The truck will visit nodes 1 and 2.  
exampleAssignments = vrv.createAssignmentsFromLocSeq2D(
                        locSeq         = locs[0:2],
                        serviceTimeSec = 30.0,
                        objectID       = 'Truck',
                        routeType      = 'fastest',
                        dataProvider   = 'OSRM-online',
                        leafletColor   = 'red')

# The truck will then pause 45 seconds before taking a route from node 4 to node 3. 
# There is an obvious break in these routes.  
exampleAssignments = vrv.createAssignmentsFromLocSeq2D(
                        initAssignments = exampleAssignments,
                        locSeq          = locs[-1:1:-1],
                        serviceTimeSec  = 30.0,
                        startTimeSec    = max(exampleAssignments['endTimeSec']) + 45,
                        objectID        = 'Truck',
                        routeType       = 'fastest',
                        dataProvider    = 'OSRM-online',
                        leafletColor    = 'green')

# The car will visit nodes 1 and 4.
exampleAssignments = vrv.createAssignmentsFromLocSeq2D(
                        initAssignments = exampleAssignments,
                        locSeq          = [locs[0], locs[3]],
                        serviceTimeSec  = 30.0,
                        objectID        = 'Car',
                        routeType       = 'fastest',
                        dataProvider    = 'OSRM-online',
                        leafletColor    = 'blue') 


Message: The origin point (lat: 42.1648, lon: -78.4293) is 32.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1565, lon: -78.4234) is 161.8 meters away from the road. You might find a gap between destination point and the route.

Message: The origin point (lat: 42.1113, lon: -78.4212) is 193.8 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1443, lon: -78.4246) is 39.0 meters away from the road. You might find a gap between destination point and the route.

Message: The origin point (lat: 42.1648, lon: -78.4293) is 32.2 meters away from the road. You might find a gap between the origin point and the route.
Message: The destination point (lat: 42.1113, lon: -78.4212) is 193.8 meters away from the road. You might find a gap between destination point and the route.


In [17]:
# Visualize the locations and assignments. 

# First, generate a `Nodes` dataframe from a list of coordinates.  
# See https://veroviz.org/docs/veroviz.generateNodes.html for other methods to generate "nodes" dataframes. 
exampleNodes = vrv.createNodesFromLocs(locs = locs)

# Now, display the nodes and assignments:
myMap = vrv.createLeaflet(nodes = exampleNodes, arcs = exampleAssignments)
myMap

In [18]:
# Specify a location:
loc = [42.1382899, -78.3887493]

In [19]:
# Add a marker for the given location:
myMap = vrv.addLeafletMarker(mapObject = myMap,
                             center    = loc, 
                             radius    = 30,
                             text      = 'Loc', 
                             fontColor = 'black')
myMap

In [20]:
# Find the nearest point to the given location for each vehicle in the assignments dataframe.  
# Note: The truck has two separate paths, so it will have two nearest points (one for each path).
closestPoints = vrv.closestPointLoc2Assignments(
    loc                = loc, 
    assignments        = exampleAssignments, 
    objectID           = None, 
    ignoreStaticPoints = True)

closestPoints

{'Truck': [{'nearestPoint': [42.157619, -78.422146],
   'distMeters': 3497.1568670344905},
  {'nearestPoint': [42.129462, -78.409906], 'distMeters': 2005.269625875047}],
 'Car': [{'nearestPoint': [42.14106, -78.400805],
   'distMeters': 1034.7894588768695}]}

In [21]:
# Plot these points on the map:
for objectID in closestPoints:
    for i in range(0, len(closestPoints[objectID])):
        myMap = vrv.addLeafletMarker(mapObject = myMap, 
                                     center    = closestPoints[objectID][i]['nearestPoint'],
                                     radius    = 14)
myMap

--- 

### 5. `closestPointLoc2Arcs()`
- Like `closestPointLoc2Assignments()`, but using an `arcs` dataframe as an input.

In [22]:
'''
We'll begin by generating some data, 
which will be used by the `closestPointLoc2Arcs()` function.
'''

# Specify 4 locations that could be visited:
locs = [[42.1648, -78.4293], 
        [42.1565, -78.4234], 
        [42.1443, -78.4246],
        [42.1113, -78.4212]]

# Now, create an `Arcs` dataframe for two vehicles 
# (a truck and a car).  

# The truck will visit nodes 1 and 2.  
exampleArcs = vrv.createArcsFromLocSeq(
                 locSeq       = locs[0:2],
                 objectID     = 'Truck',
                 leafletColor = 'red')

# The truck will then take a route from node 4 to node 3. 
# There is an obvious break in these routes.  
exampleArcs = vrv.createArcsFromLocSeq(
                initArcs     = exampleArcs,
                locSeq       = locs[-1:1:-1],
                objectID     = 'Truck',
                leafletColor = 'green')

# The car will visit nodes 1 and 4.
exampleArcs = vrv.createArcsFromLocSeq(
                initArcs     = exampleArcs,
                locSeq       = [locs[0], locs[3]],
                objectID     = 'Car',
                leafletColor = 'blue') 

In [23]:
# Visualize the locations and arcs. 

# First, generate a `Nodes` dataframe from a list of coordinates.  
# See https://veroviz.org/docs/veroviz.generateNodes.html for other methods to generate "nodes" dataframes. 
exampleNodes = vrv.createNodesFromLocs(locs = locs)

# Now, display the nodes and arcs:
myMap = vrv.createLeaflet(nodes = exampleNodes, arcs = exampleArcs)
myMap

In [24]:
# Specify a location:
loc = [42.1382899, -78.3887493]

In [25]:
# Add a marker for the given location:
myMap = vrv.addLeafletMarker(mapObject = myMap,
                             center    = loc, 
                             radius    = 30,
                             text      = 'Loc', 
                             fontColor = 'black')
myMap

In [26]:
# Find the nearest point to the given location for each vehicle in the arcs dataframe.  
# Note: The truck has two separate paths, so it will have two nearest points (one for each path).
closestPoints = vrv.closestPointLoc2Arcs(
    loc                = loc, 
    arcs               = exampleArcs, 
    objectID           = None, 
    ignoreStaticPoints = True)

closestPoints

{'Truck': [{'nearestPoint': [42.1565, -78.4234],
   'distMeters': 3506.4227801031907},
  {'nearestPoint': [42.13629558707727, -78.423775302911],
   'distMeters': 2904.118573505971}],
 'Car': [{'nearestPoint': [42.13526974820381, -78.42482906468132],
   'distMeters': 3001.490582722063}]}

In [27]:
# Plot these points on the map:
for objectID in closestPoints:
    for i in range(0, len(closestPoints[objectID])):
        myMap = vrv.addLeafletMarker(mapObject = myMap, 
                                     center    = closestPoints[objectID][i]['nearestPoint'],
                                     radius    = 14)

myMap